In [19]:
import agate

In [20]:
table = agate.Table.from_csv('test_gene2pubmed', delimiter='\t')

In [21]:
print(table)

| column    | data_type |
| --------- | --------- |
| #tax_id   | Number    |
| GeneID    | Number    |
| PubMed_ID | Number    |



The data is numeric, but we want it read as categorical, because these are ID's.

In [22]:
tester = agate.TypeTester(force={
    '#tax_id': agate.Text(),
    'GeneID': agate.Text(),
    'PubMed_ID': agate.Text()
})

table = agate.Table.from_csv('test_gene2pubmed', delimiter='\t', column_types=tester)

In [23]:
print(table)

| column    | data_type |
| --------- | --------- |
| #tax_id   | Text      |
| GeneID    | Text      |
| PubMed_ID | Text      |



In [14]:
#table_bins = table.bins('GeneID')
#table_bins.print_bars('GeneID','Count')

GeneID                     Count
[0 - 4,000,000)           14,538 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                                            
[4,000,000 - 8,000,000)   16,317 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                                       
[8,000,000 - 12,000,000)   5,219 ▓░░░░░░░░░░░░░░░                                                                       
[12,000,000 - 16,000,000)  9,097 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░                                                            
[16,000,000 - 20,000,000)  5,517 ▓░░░░░░░░░░░░░░░░                                                                      
[20,000,000 - 24,000,000)  5,834 ▓░░░░░░░░░░░░░░░░░                                                                     
[24,000,000 - 28,000,000)     69 ▓                                                                                      
[28,000,000 - 32,000,000) 26,993 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░         

**Q** What Gene is associated by most publications ?

In [34]:
geneId_dist = (table.pivot('PubMed_ID', 'GeneID' ))
#geneId_dist = geneId_dist.order_by('Count', reverse=True)
geneId_dist.print_table(max_rows=5)

| PubMed_ID | 1246500 | 1246501 | 1246502 | 1246503 | 1246504 | ... |
| --------- | ------- | ------- | ------- | ------- | ------- | --- |
| 9873079   |       1 |       1 |       1 |       1 |       1 | ... |
| 9812361   |       0 |       0 |       1 |       0 |       0 | ... |
| 10984505  |       0 |       0 |       0 |       0 |       0 | ... |
| 7608990   |       0 |       0 |       0 |       0 |       0 | ... |
| 16413149  |       0 |       0 |       0 |       0 |       0 | ... |
| ...       |     ... |     ... |     ... |     ... |     ... | ... |


In [70]:
import pandas as pd
gene2pubmed = pd.read_table("test_gene2pubmed", sep = "\t")
#gene2pubmed = pd.read_table("data/genbank-data/gene2pubmed", sep = "\t")
#gene2pubmed.groupby(['GeneID'])['PubMed_ID'].count().head(n=10) # can we sort this to get a better idea ?

In [68]:
gene_gp_pubmedCount = gene2pubmed.groupby('GeneID', as_index=False)['PubMed_ID'].count() # because pandas preserves the row index and we need it reset to be sorted
gene_gp_pubmedCount.head(n=10)

,GeneID,PubMed_ID
0,1,32
1,2,244
2,3,6
3,9,230
4,10,664
5,11,5
6,12,184
7,13,23
8,14,32
9,15,38


**NOTE** another way to reset index is using `reset_index()` like so

`gene2pubmed.groupby('GeneID')['PubMed_ID'].count().reset_index()`

In [64]:
gene_gp_pubmedCount.sort_values('PubMed_ID', ascending=False).head(n=10)

,GeneID,PubMed_ID
2729,2716540,50
6119,3244915,42
6637,3722457,42
1998,1446560,36
6379,3399421,34
7247,4246763,33
8825,6276088,32
7659,4924737,32
7602,4924680,32
1997,1446559,31


Lets change the column header so that the misleading `PubMedID` name is now `PubMed_pubs` to show that the column shows the number of PubMed publications associated with the gene

In [71]:
gene_gp_pubmedCount = gene2pubmed.groupby('GeneID', as_index=False)['PubMed_ID'].agg({'PubMed_pubs': "count"}) # because pandas preserves the row index and we need it reset to be sorted
gene_gp_pubmedCount.sort_values('PubMed_pubs', ascending=False).head(n=10)

,GeneID,PubMed_pubs
2729,2716540,50
6119,3244915,42
6637,3722457,42
1998,1446560,36
6379,3399421,34
7247,4246763,33
8825,6276088,32
7659,4924737,32
7602,4924680,32
1997,1446559,31


In [24]:
#grouping
#by_gene = table.group_by('GeneID')
#gene_totals = by_gene.aggregate([
#    ('count', agate.Count())
#])

#sorted_gene_totals = gene_totals.order_by('count', reverse=True)

#sorted_gene_totals.print_table(max_rows=5)



| GeneID  | count |
| ------- | ----- |
| 2716540 |    50 |
| 3244915 |    42 |
| 3722457 |    42 |
| 1446560 |    36 |
| 3399421 |    34 |
| ...     |   ... |


In [77]:
import sqlite3
conn = sqlite3.connect('/home/neha/Documents/whale/metagenome/HiSeq/whaleScripts/NameNode.sqlite')
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())

[('NcbiNameNode',)]


In [80]:
c.execute("PRAGMA table_info(NcbiNameNode)")
print(c.fetchall())

[(0, 'tax_id', 'INTEGER', 0, None, 0), (1, 'name_txt', 'TEXT', 0, None, 0), (2, 'unique_name', 'TEXT', 0, None, 0), (3, 'name_class', 'TEXT', 0, None, 0), (4, 'parent_tax_id', 'INTEGER', 0, None, 0), (5, 'rank', 'TEXT', 0, None, 0)]


another way is to do this via pandas

In [83]:
import pandas as pd
import sqlite3

# for single table
conn = sqlite3.connect('/home/neha/Documents/whale/metagenome/HiSeq/whaleScripts/NameNode.sqlite')
tab = pd.read_sql_query("SELECT * from NcbiNameNode", conn)


,tax_id,name_txt,unique_name,name_class,parent_tax_id,rank
0,1,all,,synonym,1,norank
1,1,root,,scientific name,1,norank
2,2,Bacteria,Bacteria <prokaryote>,scientific name,131567,superkingdom
3,2,Monera,Monera <Bacteria>,in-part,131567,superkingdom
4,2,Procaryotae,Procaryotae <Bacteria>,in-part,131567,superkingdom


In [85]:
tab.head(n=10)

,tax_id,name_txt,unique_name,name_class,parent_tax_id,rank
0,1,all,,synonym,1,norank
1,1,root,,scientific name,1,norank
2,2,Bacteria,Bacteria <prokaryote>,scientific name,131567,superkingdom
3,2,Monera,Monera <Bacteria>,in-part,131567,superkingdom
4,2,Procaryotae,Procaryotae <Bacteria>,in-part,131567,superkingdom
5,2,Prokaryota,Prokaryota <Bacteria>,in-part,131567,superkingdom
6,2,Prokaryotae,Prokaryotae <Bacteria>,in-part,131567,superkingdom
7,2,bacteria,bacteria <blast2>,blast name,131567,superkingdom
8,2,eubacteria,,genbank common name,131567,superkingdom
9,2,not Bacteria Haeckel 1894,,synonym,131567,superkingdom
